In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
!nvidia-smi

Mon Nov 22 11:55:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    69W / 149W |   1072MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/gdrive/MyDrive/Aspect-Term-Extraction-and-Analysis/data

/content/gdrive/MyDrive/Aspect-Term-Extraction-and-Analysis/data


In [ ]:
import os
os.listdir()

['laptops_test.csv',
 'restaurants_test.csv',
 'restaurants_train.csv',
 'laptops_train.csv']

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 12.7 MB/s 
     |████████████████████████████████| 895 kB 32.0 MB/s 
     |████████████████████████████████| 3.3 MB 45.5 MB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics
import torch
from transformers import BertModel,BertTokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
class AspectTermExtraction(torch.nn.Module):
    def __init__(self, pretrain_model):
        super(AspectTermExtraction, self).__init__()
        self.bert = BertModel.from_pretrained(pretrain_model)
        self.lstm = torch.nn.Linear(self.bert.config.hidden_size, 3)
        # self.lstm = torch.nn.LSTM(self.bert.config.hidden_size,3)

    def forward(self, ids, aspect, masks):
        bert_outputs,_ = self.bert(input_ids=ids, attention_mask=masks, return_dict = False)
        lstm_outputs= self.lstm(bert_outputs)
        # lstm_outputs,_= self.lstm(bert_outputs)
        if aspect is not None:
            asp, lstm_outputs = aspect.view(-1), lstm_outputs.view(-1,3)
            loss = torch.nn.CrossEntropyLoss()(lstm_outputs, asp)
            return loss
        else:
            return lstm_outputs

In [ ]:
class dataset(Dataset):
    def __init__(self, df, tokenizer):
        self.dataframe = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokens, aspect, polarity = self.dataframe.iloc[idx, :3].values
        tokens, aspect, polarity = tokens.replace("'", "").strip("][").split(', '), aspect.strip('][').split(', '), polarity.strip('][').split(', ')
        bert_tokens, bert_aspect, bert_polarity = [], [], []
        for i in range(len(tokens)):
            t = self.tokenizer.tokenize(tokens[i])
            l=len(t)
            bert_tokens += t
            ba = [int(aspect[i])]*l
            bert_aspect += ba
            bp = [int(polarity[i])]*l
            bert_polarity += bp
        bert_ids = self.tokenizer.convert_tokens_to_ids(bert_tokens)
        return bert_tokens, torch.tensor(bert_ids), torch.tensor(bert_aspect), torch.tensor(bert_polarity)

    def __len__(self):
        return len(self.dataframe)

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = AspectTermExtraction("bert-base-uncased").to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
samplez1=1500
samplez2=1000

restaurants_train, restaurants_test = dataset(pd.read_csv("restaurants_train.csv")[:samplez1], tokenizer), dataset(pd.read_csv("restaurants_test.csv")[:samplez1], tokenizer)
laptops_train, laptops_test = dataset(pd.read_csv("laptops_train.csv")[:samplez2], tokenizer), dataset(pd.read_csv("laptops_test.csv")[:samplez2], tokenizer)
train, test = ConcatDataset([laptops_train, restaurants_train]),  ConcatDataset([laptops_test, restaurants_test])

In [ ]:
def mini_batch(samples):
    lis=[]
    for i in range(3):
      a = [s[i+1] for s in samples]
      a = pad_sequence(a, batch_first=True)
      lis.append(a)
    masks = torch.zeros(lis[0].shape, dtype=torch.long)
    masks = masks.masked_fill(lis[0] != 0, 1)   

    return lis[0], lis[1], lis[2], masks

In [ ]:
train_loader = DataLoader(train, batch_size=50, collate_fn=mini_batch, shuffle = True)
test_loader = DataLoader(test, batch_size=200, collate_fn=mini_batch, shuffle = True)

In [ ]:
def train_model(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        lv, losses, correct_predictions= 0, [], 0
        for data in loader:
            ids, aspect, _, masks = data
            ids, aspect, masks = ids.to(DEVICE), aspect.to(DEVICE), masks.to(DEVICE)
            loss = model(ids=ids, aspect=aspect, masks=masks)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            lv += 1
            print('Epoch:', epoch, "\t loss:", np.mean(losses), "\t Batch:", lv, "/" , all_data)         
    


In [ ]:
train_model(train_loader, 3)

Epoch: 0 	 loss: 1.0883560180664062 	 Batch: 1 / 50
Epoch: 0 	 loss: 0.88282510638237 	 Batch: 2 / 50
Epoch: 0 	 loss: 0.7357795933882395 	 Batch: 3 / 50
Epoch: 0 	 loss: 0.6318740993738174 	 Batch: 4 / 50
Epoch: 0 	 loss: 0.5590944945812225 	 Batch: 5 / 50
Epoch: 0 	 loss: 0.5022090847293535 	 Batch: 6 / 50
Epoch: 0 	 loss: 0.4735914958374841 	 Batch: 7 / 50
Epoch: 0 	 loss: 0.44649521447718143 	 Batch: 8 / 50
Epoch: 0 	 loss: 0.43000493943691254 	 Batch: 9 / 50
Epoch: 0 	 loss: 0.4143307253718376 	 Batch: 10 / 50
Epoch: 0 	 loss: 0.40478040819818323 	 Batch: 11 / 50
Epoch: 0 	 loss: 0.3893280141055584 	 Batch: 12 / 50
Epoch: 0 	 loss: 0.37526359122533065 	 Batch: 13 / 50
Epoch: 0 	 loss: 0.36377034336328506 	 Batch: 14 / 50
Epoch: 0 	 loss: 0.35560605227947234 	 Batch: 15 / 50
Epoch: 0 	 loss: 0.3447420885786414 	 Batch: 16 / 50
Epoch: 0 	 loss: 0.337461910703603 	 Batch: 17 / 50
Epoch: 0 	 loss: 0.3314126580953598 	 Batch: 18 / 50
Epoch: 0 	 loss: 0.3244257577155766 	 Batch: 19 / 50

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/Aspect-Term-Extraction-and-Analysis/data/AspectTermExtraction.pkl')

In [ ]:
pred, y = [], []
tn=['class0','class1','class2']
with torch.no_grad():
    for i in test_loader:
        ids, aspect, _, x = i
        ids, aspect, x= ids.to(DEVICE), aspect.to(DEVICE), x.to(DEVICE)
        dimension=2
        _, preds = torch.max(model(ids=ids, aspect=None, masks=x), dim=dimension)
        pred += list([int(j) for k in preds for j in k ])
        y += list([int(j) for k in aspect for j in k ])
result=classification_report(pred, y, target_names=tn)
print(result)

              precision    recall  f1-score   support

      class0       1.00      0.99      0.99    137879
      class1       0.74      0.82      0.77      5144
      class2       0.66      0.82      0.73      2557

    accuracy                           0.98    145580
   macro avg       0.80      0.87      0.83    145580
weighted avg       0.98      0.98      0.98    145580



In [ ]:
os.getcwd()

'/content/gdrive/My Drive/Aspect-Term-Extraction-and-Analysis/data'

In [ ]:
PATH='/content/gdrive/My Drive/Aspect-Term-Extraction-and-Analysis/data/AspectTermExtraction.pkl'
model = AspectTermExtraction("bert-base-uncased")
model.load_state_dict(torch.load(PATH))
model = model.to(DEVICE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pred, y = [], []
tn=['class0','class1','class2']
with torch.no_grad():
    for i in test_loader:
        ids, aspect, _, x = i
        ids, aspect, x= ids.to(DEVICE), aspect.to(DEVICE), x.to(DEVICE)
        dimension=2
        _, preds = torch.max(model(ids=ids, aspect=None, masks=x), dim=dimension)
        pred += list([int(j) for k in preds for j in k ])
        y += list([int(j) for k in aspect for j in k ])
result=classification_report(pred, y, target_names=tn)
print(result)

              precision    recall  f1-score   support

      class0       1.00      0.99      0.99    140199
      class1       0.74      0.82      0.77      5144
      class2       0.66      0.82      0.73      2557

    accuracy                           0.98    147900
   macro avg       0.80      0.87      0.83    147900
weighted avg       0.98      0.98      0.98    147900

